In [1]:
import sys
sys.path.append("..")

from utils.evaluation import *
from utils.utils import *

from data import dataset_preprocessing

from utils.evaluation import get_metrics
from xgboost import XGBClassifier, XGBRegressor
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso

from scipy import stats

import pandas as pd
import numpy as np
import os

import pickle

In [2]:
dataset_name = "xAPI-Edu-Data"
mode="cv"
RS=68
hct=10
test_ratio=0.2
val_ratio=0.1
folds=5
target = "categorical"
experiment_name = "EDM_results"

### Describe raw data

In [3]:
df = pd.read_csv(f"../data/raw/{dataset_name}/{dataset_name}.csv",sep=",")


In [4]:
y_col = "Class"
demographic_cols = ["gender", "NationalITy", "PlaceofBirth", "Relation"]
perf_cols = []
activity_cols = ["raisedhands", "VisITedResources", "AnnouncementsView", "Discussion", 'StudentAbsenceDays']
other_cols = ['GradeID', 'ParentAnsweringSurvey', 'ParentschoolSatisfaction', 'SectionID', 'Semester', 'StageID', "Topic"]
set(df.columns)-set([y_col]+demographic_cols+perf_cols+activity_cols+other_cols)

set()

In [5]:
desc_df_dict = {"No. of samples": df.shape[0],
           "No. of features": df.shape[1],
           "Performance features": len(perf_cols),
           "Demographic features": len(demographic_cols),
           "Activity features": len(activity_cols),
           "Other features": len(other_cols),
           "Categorical features": len(df.columns[list(np.logical_and(df.nunique() > 2, df.dtypes == "object"))]),     
           "Total cardinality": df[df.columns[list(np.logical_and(df.nunique() > 2, df.dtypes == "object"))]].nunique().sum(),     
           "% NA": df.isna().sum().sum()/sum(df.shape),
           "Target $\textbf{y} \in$": f"[1..{df[y_col].nunique()}]",
#            "High cardinality levels":  list(df.loc[:,list(df.columns[list(np.logical_and(df.nunique() >= 10, df.dtypes == "object"))])].nunique().sort_values().values),
          
}
desc_df = pd.DataFrame([desc_df_dict],index=["cortez"])
desc_df

,No. of samples,No. of features,Performance features,Demographic features,Activity features,Other features,Categorical features,Total cardinality,% NA,Target $\textbf{y} \in$
cortez,480,17,0,4,5,7,7,59,0.0,[1..3]


In [6]:
print(desc_df.transpose().to_latex())

\begin{tabular}{ll}
\toprule
{} &  cortez \\
\midrule
No. of samples          &     480 \\
No. of features         &      17 \\
Performance features    &       0 \\
Demographic features    &       4 \\
Activity features       &       5 \\
Other features          &       7 \\
Categorical features    &       7 \\
Total cardinality       &      59 \\
\% NA                    &       0 \\
Target \$\textbackslash textbf\{y\} \textbackslash in\$ &  [1..3] \\
\bottomrule
\end{tabular}



### Preprocessing and preparation

In [7]:
data_path = f"{mode}_RS{RS}_hct{hct}"
if mode == "cv":
    data_path += f"_{folds}folds"
elif mode == "train_test":
    data_path += f"_split{1-test_ratio*100}-{test_ratio*100}"
elif mode == "train_val_test":
    data_path += f"_split{round(100-(test_ratio+val_ratio)*100)}-{round(test_ratio*100)}-{round(val_ratio*100)}"


# If no data_dict for the configuration exists, run preprocessing, else load data_dict
if not os.path.exists(f"../data/prepared/{dataset_name}/"+data_path+"/data_dict.pickle"):
    dataset_preprocessing.process_dataset(dataset_name, target, mode, RS, hct, test_ratio, val_ratio, folds)
with open(f"../data/prepared/{dataset_name}/{data_path}/data_dict.pickle", 'rb') as handle:
        data_dict = pickle.load(handle)


## Evaluation of categorical data treatment methods

In [8]:
conditions = ["ignore", "ohe", "target", "ordinal", "catboost", "glmm"]

In [9]:
early_stopping_rounds = 10
max_evals = 50

In [10]:
if not os.path.exists(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle"):

    results_encodings = {}
    results_encodings_feature_importances = {}

    for fold in range(folds):
        results_encodings[fold] = {}
        results_encodings_feature_importances[fold] = {}
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = data_dict[f"y_test_{fold}"]
        y_train_val = np.concatenate([y_train,y_val])

        u,c = np.unique(y_train_val,return_counts=True)
        nb_classes = len(u)
        baseline = np.argmax(c)

        y_train_val_pred_base = np.ones(y_train_val.shape[0])*baseline
        y_test_pred_base = np.ones(y_test.shape[0])*baseline

        results_encodings[fold]["Baseline"] = {}
        eval_res_train = get_metrics(get_one_hot(y_train_val, nb_classes), get_one_hot(y_train_val_pred_base.astype(int), nb_classes), target=target)
        for metric in eval_res_train.keys():
            results_encodings[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(get_one_hot(y_test, nb_classes), get_one_hot(y_test_pred_base.astype(int), nb_classes), target=target)
        for metric in eval_res_test.keys():
            results_encodings[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]


        for condition in conditions:
            print(f"Preparing results for fold {fold}, condition={condition}")
            # Retrieve data
            z_cols = data_dict["z_cols"]

            X_train = data_dict[f"X_train_{fold}"]
            y_train = data_dict[f"y_train_{fold}"]

            X_val = data_dict[f"X_val_{fold}"]
            y_val = data_dict[f"y_val_{fold}"]

            X_test = data_dict[f"X_test_{fold}"]
            y_test = data_dict[f"y_test_{fold}"]

     #         Define condition data subset
            if condition != "ignore":
                z_encoded_train = data_dict[f"z_{condition}_encoded_train_{fold}"]
                z_encoded_val = data_dict[f"z_{condition}_encoded_val_{fold}"] 
                z_encoded_test = data_dict[f"z_{condition}_encoded_test_{fold}"] 

                X_train = pd.concat([X_train,z_encoded_train],axis=1)
                X_val = pd.concat([X_val,z_encoded_val],axis=1)
                X_test = pd.concat([X_test,z_encoded_test],axis=1)

            
            X_train_val = pd.concat([X_train,X_val])
            y_train_val = np.concatenate([y_train,y_val])

            # Train base models
            res, feats = evaluate_logreg(X_train_val, y_train_val, X_test, y_test, target=target,tune=False, seed=RS)
            results_encodings[fold]["LR_"+condition] = res
            results_encodings_feature_importances[fold]["LR_"+condition] = feats

            res, feats = evaluate_xgb(X_train_val, y_train_val, X_test, y_test, target, tune=False, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_encodings[fold]["XGB_"+condition] = res
            results_encodings_feature_importances[fold]["XGB_"+condition] = feats

            # Train tuned models
            res, feats = evaluate_logreg(X_train_val, y_train_val, X_test, y_test, target=target, max_evals=max_evals, tune=True, seed=RS)
            results_encodings[fold]["LR_"+condition+"_tuned"] = res
            results_encodings_feature_importances[fold]["LR_"+condition+"_tuned"] = feats

            res, feats = evaluate_xgb(X_train_val, y_train_val, X_test, y_test, target, tune=True, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_encodings[fold]["XGB_"+condition+"_tuned"] = res
            results_encodings_feature_importances[fold]["XGB_"+condition+"_tuned"] = feats
    
    if not os.path.exists(f"../results/{dataset_name}/{experiment_name}"):
        os.makedirs(f"../results/{dataset_name}/{experiment_name}")
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle", 'wb') as handle:
        pickle.dump(results_encodings, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings_feature_importances.pickle", 'wb') as handle:
        pickle.dump(results_encodings_feature_importances, handle, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings.pickle", 'rb') as handle:
        results_encodings = pickle.load(handle)
    with open(f"../results/{dataset_name}/{experiment_name}/results_encodings_feature_importances.pickle", 'rb') as handle:
        results_encodings_feature_importances = pickle.load(handle)
        
        
results_encodings_df = pd.DataFrame(results_encodings[0]).transpose().sort_values("F1 Test",ascending=False).round(4)
results_encodings_df[["Accuracy Train", "F1 Train", "AUROC Train", "Accuracy Test", "F1 Test", "AUROC Test"]].style.highlight_max(color = 'lightgreen', axis = 0)

,Accuracy Train,F1 Train,AUROC Train,Accuracy Test,F1 Test,AUROC Test
XGB_ordinal,1.000000,1.000000,1.000000,0.739600,0.757400,0.878200
XGB_glmm_tuned,0.997400,0.997400,1.000000,0.739600,0.755300,0.865500
XGB_ohe,1.000000,1.000000,1.000000,0.729200,0.745800,0.873100
XGB_glmm,1.000000,1.000000,1.000000,0.718800,0.740800,0.878800
XGB_ignore,1.000000,1.000000,1.000000,0.718800,0.737700,0.842200
XGB_ignore_tuned,1.000000,1.000000,1.000000,0.718800,0.737700,0.842800
XGB_ohe_tuned,1.000000,1.000000,1.000000,0.708300,0.735400,0.866200
LR_ohe_tuned,0.859400,0.862500,0.952100,0.718800,0.731300,0.879500
XGB_target_tuned,0.971400,0.971400,0.998900,0.697900,0.722400,0.854700
XGB_target,1.000000,1.000000,1.000000,0.697900,0.721500,0.846900


In [11]:
results_encodings_df = pd.DataFrame(results_encodings[1]).transpose().sort_values("F1 Test",ascending=False).round(4)
results_encodings_df[["Accuracy Train", "F1 Train", "AUROC Train", "Accuracy Test", "F1 Test", "AUROC Test"]].style.highlight_max(color = 'lightgreen', axis = 0)

,Accuracy Train,F1 Train,AUROC Train,Accuracy Test,F1 Test,AUROC Test
LR_target,0.791700,0.800100,0.924600,0.770800,0.771000,0.901700
LR_target_tuned,0.791700,0.800100,0.924600,0.770800,0.771000,0.901700
LR_ignore_tuned,0.763000,0.773000,0.908500,0.770800,0.767500,0.912200
LR_ignore,0.763000,0.773000,0.908500,0.770800,0.767500,0.912200
LR_glmm_tuned,0.781200,0.790400,0.919400,0.750000,0.752700,0.895300
XGB_glmm_tuned,0.872400,0.876300,0.966100,0.750000,0.750000,0.898800
LR_catboost_tuned,0.776000,0.785900,0.914300,0.750000,0.747200,0.890900
LR_catboost,0.778600,0.788600,0.914600,0.750000,0.747200,0.890200
XGB_target,1.000000,1.000000,1.000000,0.739600,0.743700,0.898600
XGB_ordinal_tuned,0.974000,0.974000,0.997900,0.739600,0.743500,0.901700


### Performance Comparison

In [12]:
# For LR
models = ["Baseline"]+[i for i in results_encodings[0].keys() if ("tuned" in i and "LR" in i)]
metric = "F1 Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())

df_mean = pd.DataFrame((use_df).mean(axis=0).round(2).astype(str) + " (" + use_df.std(axis=0).round(2).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_lr = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_lr.style.apply(negative_bold)


,Baseline,LR_ignore_tuned,LR_ohe_tuned,LR_target_tuned,LR_ordinal_tuned,LR_catboost_tuned,LR_glmm_tuned
0,0.2 (0.02),0.75 (0.06),0.76 (0.03),0.75 (0.06),0.73 (0.05),0.73 (0.09),0.75 (0.06)


In [13]:
# For LR
models = ["Baseline"]+[i for i in results_encodings[0].keys() if ("tuned" in i and "XGB" in i)]
metric = "F1 Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_encodings[fold_num]).loc[metric,models] for fold_num in results_encodings.keys()],index=results_encodings.keys())

df_mean = pd.DataFrame((use_df).mean(axis=0).round(2).astype(str) + " (" + use_df.std(axis=0).round(2).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_xgb = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_xgb.style.apply(negative_bold)


,Baseline,XGB_ignore_tuned,XGB_ohe_tuned,XGB_target_tuned,XGB_ordinal_tuned,XGB_catboost_tuned,XGB_glmm_tuned
0,0.2 (0.02),0.76 (0.05),0.78 (0.04),0.78 (0.06),0.78 (0.08),0.72 (0.07),0.76 (0.02)


In [14]:
res_df_lr.columns = [i.split("_")[1] if i != "Baseline" else "Baseline" for i in res_df_lr.columns]    
res_df_xgb.columns = [i.split("_")[1] if i != "Baseline" else "Baseline" for i in res_df_xgb.columns]    

latex_df_encodings = pd.concat([res_df_lr,res_df_xgb],axis=0)
latex_df_encodings.index = ["LR", "XGB"]
latex_df_encodings

,Baseline,ignore,ohe,target,ordinal,catboost,glmm
LR,0.2 (0.02),0.75 (0.06),0.76 (0.03),0.75 (0.06),0.73 (0.05),0.73 (0.09),0.75 (0.06)
XGB,0.2 (0.02),0.76 (0.05),0.78 (0.04),0.78 (0.06),0.78 (0.08),0.72 (0.07),0.76 (0.02)


In [15]:
print(latex_df_encodings.round(2).to_latex())


\begin{tabular}{llllllll}
\toprule
{} &    Baseline &       ignore &          ohe &       target &      ordinal &     catboost &         glmm \\
\midrule
LR  &  0.2 (0.02) &  0.75 (0.06) &  0.76 (0.03) &  0.75 (0.06) &  0.73 (0.05) &  0.73 (0.09) &  0.75 (0.06) \\
XGB &  0.2 (0.02) &  0.76 (0.05) &  0.78 (0.04) &  0.78 (0.06) &  0.78 (0.08) &  0.72 (0.07) &  0.76 (0.02) \\
\bottomrule
\end{tabular}



## Data Subset Comparisons

In [16]:
subsets = {"demo_only": demographic_cols,
           "perfact_only": perf_cols+activity_cols,
           "perfact_and_demo": perf_cols+activity_cols+demographic_cols,
           "all": list(df.columns)
          }

In [17]:
if not os.path.exists(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle"):

    results_subsets = {}
    results_subsets_feature_importances = {}

    for fold in range(folds):
        results_subsets[fold] = {}
        results_subsets_feature_importances[fold] = {}
        # Create baseline
        y_train = data_dict[f"y_train_{fold}"]
        y_val = data_dict[f"y_val_{fold}"]
        y_test = data_dict[f"y_test_{fold}"]
        y_train_val = np.concatenate([y_train,y_val])

        u,c = np.unique(y_train_val,return_counts=True)
        nb_classes = len(u)
        baseline = np.argmax(c)

        y_train_val_pred_base = np.ones(y_train_val.shape[0])*baseline
        y_test_pred_base = np.ones(y_test.shape[0])*baseline

        results_subsets[fold]["Baseline"] = {}
        eval_res_train = get_metrics(get_one_hot(y_train_val, nb_classes), get_one_hot(y_train_val_pred_base.astype(int), nb_classes), target=target)
        for metric in eval_res_train.keys():
            results_subsets[fold]["Baseline"][metric + " Train"] = eval_res_train[metric]
        eval_res_test = get_metrics(get_one_hot(y_test, nb_classes), get_one_hot(y_test_pred_base.astype(int), nb_classes), target=target)
        for metric in eval_res_test.keys():
            results_subsets[fold]["Baseline"][metric + " Test"] = eval_res_test[metric]


        for subset_key in subsets:
            print(f"Preparing results for fold {fold}, subset={subset_key}")
            # Retrieve data
            z_cols = data_dict["z_cols"]

            X_train = data_dict[f"X_train_{fold}"]
            y_train = data_dict[f"y_train_{fold}"]

            X_val = data_dict[f"X_val_{fold}"]
            y_val = data_dict[f"y_val_{fold}"]

            X_test = data_dict[f"X_test_{fold}"]
            y_test = data_dict[f"y_test_{fold}"]
        
            y_train_val = np.concatenate([y_train,y_val])

            # Define data subset for LR
            z_target_encoded_train = data_dict[f"z_target_encoded_train_{fold}"] 
            z_target_encoded_val = data_dict[f"z_target_encoded_val_{fold}"] 
            z_target_encoded_test = data_dict[f"z_target_encoded_test_{fold}"] 
            X_train_lr = pd.concat([X_train,z_target_encoded_train],axis=1)
            X_val_lr = pd.concat([X_val,z_target_encoded_val],axis=1)
            X_test_lr = pd.concat([X_test,z_target_encoded_test],axis=1)      
            X_train_val_lr = pd.concat([X_train_lr,X_val_lr])

            # Rescale GLMM
            for col in z_target_encoded_train.columns:
                z_mean = X_train_val_lr[col].mean()
                z_std = X_train_val_lr[col].std()
            
                X_train_val_lr[col] = (X_train_val_lr[col]-z_mean)/z_std
                X_test_lr[col] = (X_test_lr[col]-z_mean)/z_std
            
            
            # Define data subset for XGB
            z_ordinal_encoded_train = data_dict[f"z_ordinal_encoded_train_{fold}"] 
            z_ordinal_encoded_val = data_dict[f"z_ordinal_encoded_val_{fold}"] 
            z_ordinal_encoded_test = data_dict[f"z_ordinal_encoded_test_{fold}"] 
            X_train_xgb = pd.concat([X_train,z_ordinal_encoded_train],axis=1)
            X_val_xgb = pd.concat([X_val,z_ordinal_encoded_val],axis=1)
            X_test_xgb = pd.concat([X_test,z_ordinal_encoded_test],axis=1)
            X_train_val_xgb = pd.concat([X_train_xgb,X_val_xgb])


            # Define data subset for evaluation
            X_train_val_lr = X_train_val_lr[[i for i in X_train_val_lr.columns if any([j in i for j in subsets[subset_key]])]]
            X_test_lr = X_test_lr[[i for i in X_test_lr.columns if any([j in i for j in subsets[subset_key]])]]
            X_train_val_xgb = X_train_val_xgb[[i for i in X_train_val_xgb.columns if any([j in i for j in subsets[subset_key]])]]
            X_test_xgb = X_test_xgb[[i for i in X_test_xgb.columns if any([j in i for j in subsets[subset_key]])]]


            # Train base models
            res, feats = evaluate_logreg(X_train_val_lr, y_train_val, X_test_lr, y_test, target=target,tune=False, seed=RS)
            results_subsets[fold]["LR_"+subset_key] = res
            results_subsets_feature_importances[fold]["LR_"+subset_key] = feats

            res, feats = evaluate_xgb(X_train_val_xgb, y_train_val, X_test_xgb, y_test, target, tune=False, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_subsets[fold]["XGB_"+subset_key] = res
            results_subsets_feature_importances[fold]["XGB_"+subset_key] = feats

            # Train tuned models
            res, feats = evaluate_logreg(X_train_val_lr, y_train_val, X_test_lr, y_test, target=target, max_evals=max_evals, tune=True, seed=RS)
            results_subsets[fold]["LR_"+subset_key+"_tuned"] = res
            results_subsets_feature_importances[fold]["LR_"+subset_key+"_tuned"] = feats

            res, feats = evaluate_xgb(X_train_val_xgb, y_train_val, X_test_xgb, y_test, target, tune=True, max_evals=max_evals, early_stopping_rounds=early_stopping_rounds, seed=RS)
            results_subsets[fold]["XGB_"+subset_key+"_tuned"] = res
            results_subsets_feature_importances[fold]["XGB_"+subset_key+"_tuned"] = feats
    
    if not os.path.exists(f"../results/{dataset_name}/{experiment_name}"):
        os.makedirs(f"../results/{dataset_name}/{experiment_name}")
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle", 'wb') as handle:
        pickle.dump(results_subsets, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets_feature_importances.pickle", 'wb') as handle:
        pickle.dump(results_subsets_feature_importances, handle, protocol=pickle.HIGHEST_PROTOCOL)

else:
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets.pickle", 'rb') as handle:
        results_subsets = pickle.load(handle)
    with open(f"../results/{dataset_name}/{experiment_name}/results_subsets_feature_importances.pickle", 'rb') as handle:
        results_subsets_feature_importances = pickle.load(handle)
        
        
results_subsets_df = pd.DataFrame(results_subsets[0]).transpose().sort_values("F1 Test",ascending=False).round(4)
results_subsets_df[["Accuracy Train", "F1 Train", "AUROC Train", "Accuracy Test", "F1 Test", "AUROC Test"]].style.highlight_max(color = 'lightgreen', axis = 0)

,Accuracy Train,F1 Train,AUROC Train,Accuracy Test,F1 Test,AUROC Test
XGB_perfact_and_demo,1.000000,1.000000,1.000000,0.739600,0.760500,0.861500
XGB_perfact_and_demo_tuned,0.974000,0.973900,0.998600,0.739600,0.758400,0.860200
XGB_all,1.000000,1.000000,1.000000,0.739600,0.757400,0.878200
XGB_all_tuned,0.966100,0.966300,0.995600,0.718800,0.738100,0.857700
XGB_perfact_only,0.997400,0.997500,1.000000,0.697900,0.726000,0.828500
XGB_perfact_only_tuned,0.908900,0.910800,0.978200,0.687500,0.710300,0.826200
LR_perfact_only,0.744800,0.753700,0.894300,0.677100,0.698000,0.843500
LR_perfact_only_tuned,0.744800,0.753700,0.894300,0.677100,0.698000,0.843500
LR_all,0.843800,0.847000,0.938300,0.635400,0.650600,0.831500
LR_all_tuned,0.843800,0.847000,0.936800,0.635400,0.650600,0.829300


In [18]:
results_subsets_df = pd.DataFrame(results_subsets[1]).transpose().sort_values("F1 Test",ascending=False).round(4)
results_subsets_df[["Accuracy Train", "F1 Train", "AUROC Train", "Accuracy Test", "F1 Test", "AUROC Test"]].style.highlight_max(color = 'lightgreen', axis = 0)

,Accuracy Train,F1 Train,AUROC Train,Accuracy Test,F1 Test,AUROC Test
XGB_all_tuned,0.989600,0.989600,0.999300,0.760400,0.761100,0.900500
LR_all,0.796900,0.805400,0.926100,0.750000,0.753700,0.893000
LR_perfact_only_tuned,0.731800,0.745300,0.883200,0.750000,0.751600,0.887000
LR_perfact_only,0.731800,0.745300,0.883200,0.750000,0.751600,0.887000
LR_all_tuned,0.794300,0.802800,0.925700,0.739600,0.744200,0.891000
LR_perfact_and_demo_tuned,0.765600,0.775100,0.908000,0.739600,0.740100,0.895700
LR_perfact_and_demo,0.765600,0.775100,0.908000,0.739600,0.740100,0.895700
XGB_perfact_only_tuned,0.812500,0.822300,0.930900,0.729200,0.729500,0.884600
XGB_all,1.000000,1.000000,1.000000,0.718800,0.721700,0.877700
XGB_perfact_and_demo,0.997400,0.997600,1.000000,0.718800,0.719900,0.872000


### Performance Results

In [19]:
# For LR
models = ["Baseline"]+[i for i in results_subsets[0].keys() if ("tuned" in i and "LR" in i)]
metric = "F1 Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())

df_mean = pd.DataFrame((use_df).mean(axis=0).round(2).astype(str) + " (" + use_df.std(axis=0).round(2).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_lr = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_lr.style.apply(negative_bold)


,Baseline,LR_demo_only_tuned,LR_perfact_only_tuned,LR_perfact_and_demo_tuned,LR_all_tuned
0,0.2 (0.02),0.39 (0.04),0.74 (0.03),0.74 (0.05),0.74 (0.06)


In [20]:
# For XGB
models = ["Baseline"]+[i for i in results_subsets[0].keys() if ("tuned" in i and "XGB" in i)]
metric = "F1 Test"

#####
dataset_res_dict = {}
best_models = {}
t_test_results = {}

use_df = pd.DataFrame([pd.DataFrame(results_subsets[fold_num]).loc[metric,models] for fold_num in results_subsets.keys()],index=results_subsets.keys())

df_mean = pd.DataFrame((use_df).mean(axis=0).round(2).astype(str) + " (" + use_df.std(axis=0).round(2).astype(str) + ")").transpose()
model_dict = {i: df_mean[i].values[0] for i in df_mean.columns}

best_model = use_df.columns[use_df.mean(axis=0).argmax()]

t_test_res = np.array([stats.ttest_rel(use_df[best_model].values, use_df[model].values)[1] for model in models]).round(3)
t_test_res[np.isnan(t_test_res)] = 1.
    
res_df_xgb = pd.DataFrame([model_dict])

def negative_bold(val):
    i = np.where(val.name==np.array(models))[0][0]
    return ["font-weight: bold"  if t_test_res[i]>=0.05 else "" for dataset_name in val.keys()]
    # Case without transpose:
#     return ["font-weight: bold"  if t_test_results[val.name][i]>=0.05 else "" for i in range(len(val))]

res_df_xgb.style.apply(negative_bold)


,Baseline,XGB_demo_only_tuned,XGB_perfact_only_tuned,XGB_perfact_and_demo_tuned,XGB_all_tuned
0,0.2 (0.02),0.54 (0.03),0.74 (0.03),0.75 (0.05),0.78 (0.05)


In [21]:
res_df_lr.columns = [i[3:-6] if i != "Baseline" else "Baseline" for i in res_df_lr.columns]    
res_df_xgb.columns = [i[4:-6] if i != "Baseline" else "Baseline" for i in res_df_xgb.columns]    

latex_df_subsets = pd.concat([res_df_lr,res_df_xgb],axis=0)
latex_df_subsets.index = ["LR", "XGB"]
latex_df_subsets

,Baseline,demo_only,perfact_only,perfact_and_demo,all
LR,0.2 (0.02),0.39 (0.04),0.74 (0.03),0.74 (0.05),0.74 (0.06)
XGB,0.2 (0.02),0.54 (0.03),0.74 (0.03),0.75 (0.05),0.78 (0.05)


In [22]:
print(latex_df_subsets.round(2).transpose().to_latex())


\begin{tabular}{lll}
\toprule
{} &           LR &          XGB \\
\midrule
Baseline         &   0.2 (0.02) &   0.2 (0.02) \\
demo\_only        &  0.39 (0.04) &  0.54 (0.03) \\
perfact\_only     &  0.74 (0.03) &  0.74 (0.03) \\
perfact\_and\_demo &  0.74 (0.05) &  0.75 (0.05) \\
all              &  0.74 (0.06) &  0.78 (0.05) \\
\bottomrule
\end{tabular}



### Feature Importance

In [23]:
# top_10_importances = {}

# for model in list(results_subsets_feature_importances[fold].keys()):
#     imp_df = pd.concat([results_subsets_feature_importances[fold][model] for fold in range(folds)],axis=1)

#     if "LR" in model:
#         direction = imp_df.apply(lambda x: np.sign(x))
#         imp_df = imp_df.abs()

#     imp_df = imp_df/imp_df.sum(axis=0)

#     mean_imp_df = imp_df.mean(axis=1)
#     std_imp_df = imp_df.std(axis=1)

#     mean_imp_df = mean_imp_df.sort_values(ascending=False)
#     std_imp_df = std_imp_df.loc[mean_imp_df.index]
#     final_imps = mean_imp_df[:10]
#     final_imps["Rest"] = sum(mean_imp_df[10:])
#     top_5_importances[model] = np.array([final_imps.index.values, final_imps.values])

In [24]:
demo_importances = {}
demo_importances_stds = {}

for model in list(results_subsets_feature_importances[0].keys()):
    if "demo" in model or "all" in model:
        imp_df_all = pd.concat([results_subsets_feature_importances[fold][model] for fold in range(folds)],axis=1)
        
        if "LR" in model:
            direction = imp_df_all.apply(lambda x: np.sign(x))
            imp_df_all = imp_df_all.abs()
        if imp_df_all.sum().sum()!=0:
            imp_df = imp_df_all/imp_df_all.sum(axis=0)
        imp_df = imp_df.fillna(1/imp_df.shape[0])
#         imp_df = imp_df.loc[demographic_cols]

#         mean_imp_df = imp_df.mean(axis=1)
#         std_imp_df = imp_df.std(axis=1)

#         mean_imp_df = mean_imp_df.sort_values(ascending=False)
#         std_imp_df = std_imp_df.loc[mean_imp_df.index]
#         final_imps = mean_imp_df#[:10]
#         final_imps["Rest"] = sum(mean_imp_df[10:])
#         final_imps["Total"] = sum(mean_imp_df)
        demo_importances[model] = np.round(np.mean(imp_df.loc[[i for i in imp_df.index if any([j in i for j in demographic_cols])]].sum(axis=0)),2)#final_imps.values
        demo_importances_stds[model] = np.round(np.std(imp_df.loc[[i for i in imp_df.index if any([j in i for j in demographic_cols])]].sum(axis=0)),2)#final_imps.values


In [25]:
lr_demo_imp = pd.Series({i: demo_importances[i] for i in demo_importances if "LR" in i and "tuned" in i})
xgb_demo_imp = pd.Series({i: demo_importances[i] for i in demo_importances if "XGB" in i and "tuned" in i})
lr_demo_imp.index = [i[3:-6] for i in lr_demo_imp.index]    
xgb_demo_imp.index = [i[4:-6] for i in xgb_demo_imp.index]    

lr_demo_imp_stds = pd.Series({i: demo_importances_stds[i] for i in demo_importances_stds if "LR" in i and "tuned" in i})
xgb_demo_imp_stds = pd.Series({i: demo_importances_stds[i] for i in demo_importances_stds if "XGB" in i and "tuned" in i})
lr_demo_imp_stds.index = [i[3:-6] for i in lr_demo_imp_stds.index]    
xgb_demo_imp_stds.index = [i[4:-6] for i in xgb_demo_imp_stds.index]    


latex_df_imp = pd.DataFrame([lr_demo_imp.astype(str) + " (" + lr_demo_imp_stds.astype(str) + ")",
                             xgb_demo_imp.astype(str) + " (" + xgb_demo_imp_stds.astype(str) + ")"])
latex_df_imp.index = ["LR", "XGB"]
latex_df_imp

,demo_only,perfact_and_demo,all
LR,1.0 (0.0),0.32 (0.14),0.23 (0.1)
XGB,1.0 (0.0),0.26 (0.03),0.21 (0.02)


In [26]:
print(latex_df_imp.transpose().to_latex())

\begin{tabular}{lll}
\toprule
{} &           LR &          XGB \\
\midrule
demo\_only        &    1.0 (0.0) &    1.0 (0.0) \\
perfact\_and\_demo &  0.32 (0.14) &  0.26 (0.03) \\
all              &   0.23 (0.1) &  0.21 (0.02) \\
\bottomrule
\end{tabular}



### Mean impact of using demo features

In [27]:
np.random.seed(RS)

def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0) # only difference

mean_abs_differences = []
for fold in range(folds):
    X_train = data_dict[f"X_train_{fold}"]
    y_train = data_dict[f"y_train_{fold}"]

    X_val = data_dict[f"X_val_{fold}"]
    y_val = data_dict[f"y_val_{fold}"]

    X_test = data_dict[f"X_test_{fold}"]
    y_test = data_dict[f"y_test_{fold}"]

    y_train_val = np.concatenate([y_train,y_val])

    # Define data subset for LR
    z_glmm_encoded_train = data_dict[f"z_glmm_encoded_train_{fold}"] 
    z_glmm_encoded_val = data_dict[f"z_glmm_encoded_val_{fold}"] 
    z_glmm_encoded_test = data_dict[f"z_glmm_encoded_test_{fold}"] 

    X_train_lr = pd.concat([X_train,z_glmm_encoded_train],axis=1)
    X_val_lr = pd.concat([X_val,z_glmm_encoded_val],axis=1)
    X_test_lr = pd.concat([X_test,z_glmm_encoded_test],axis=1)      
    X_train_val_lr = pd.concat([X_train_lr,X_val_lr])

    # Rescale GLMM
    for col in z_glmm_encoded_train.columns:
        z_mean = X_train_val_lr[col].mean()
        z_std = X_train_val_lr[col].std()

        X_train_val_lr[col] = (X_train_val_lr[col]-z_mean)/z_std
        X_test_lr[col] = (X_test_lr[col]-z_mean)/z_std


    # Define data subset for XGB
    z_ordinal_encoded_train = data_dict[f"z_ordinal_encoded_train_{fold}"] 
    z_ordinal_encoded_val = data_dict[f"z_ordinal_encoded_val_{fold}"] 
    z_ordinal_encoded_test = data_dict[f"z_ordinal_encoded_test_{fold}"] 
    X_train_xgb = pd.concat([X_train,z_ordinal_encoded_train],axis=1)
    X_val_xgb = pd.concat([X_val,z_ordinal_encoded_val],axis=1)
    X_test_xgb = pd.concat([X_test,z_ordinal_encoded_test],axis=1)
    X_train_val_xgb = pd.concat([X_train_xgb,X_val_xgb])

    final_hyperparameters = tune_logreg_multiclass(X_train_val_lr, y_train_val, max_evals=max_evals, seed=RS)
    lr = LogisticRegression(penalty="l2",
                                       solver="lbfgs",
                                       C=final_hyperparameters["C"],
                                       max_iter=10000,
                                       random_state=RS
                                       )
#     lr = Lasso(alpha=0.001)
    lr.fit(X_train_val_lr.values,y_train_val)
    y_pred_logits = softmax(lr.predict_proba(X_test_lr.values))
    y_pred = lr.predict(X_test_lr.values)

    is_not_demo = [i not in demographic_cols for i in X_train_val_lr.columns]
    y_pred_logits_notdemo = softmax(np.dot(X_test_lr.loc[:,is_not_demo],lr.coef_.transpose()[is_not_demo]))
    y_pred_notdemo = np.argmax(y_pred_logits_notdemo,axis=1)

    print(f"% different predictions w\o Demo: {np.mean(y_pred!=y_pred_notdemo)}")
    print(f"Mean absolute % difference w\o Demo: {np.mean(np.abs(y_pred_logits-y_pred_logits_notdemo))}")
    print(f"RMSE Difference w\o Demo: {np.sqrt(np.mean(np.power(y_pred-y_pred_notdemo,2)))}")
    
    mean_abs_differences.append(np.mean(y_pred!=y_pred_notdemo))

SCORE: 0.6180679737814163                             
SCORE: 0.6124630854423423                                                       
SCORE: 0.6141106139287946                                                       
SCORE: 0.6034275978665681                                                       
SCORE: 0.6043352985831933                                                       
SCORE: 0.6136691277288684                                                       
SCORE: 0.6016865653326727                                                       
SCORE: 0.6185262837908166                                                       
SCORE: 0.606569620992403                                                        
SCORE: 0.6150924789634374                                                       
SCORE: 0.6100399365759289                                                       
SCORE: 0.605845118317888                                                         
SCORE: 0.6215510816385404                            

SCORE: 0.6099117498068063                                                        
SCORE: 0.6506104801576391                                                        
SCORE: 0.5999575931735139                                                        
100%|██████████| 50/50 [00:04<00:00, 12.24trial/s, best loss: 0.5999575931735139]
The best hyperparameters are :  

{'C': 0.35779918592826215}
% different predictions w\o Demo: 0.0625
Mean absolute % difference w\o Demo: 0.008221464018610977
RMSE Difference w\o Demo: 0.46770717334674267
SCORE: 0.6599628223631345                             
SCORE: 0.6261928283281961                             
SCORE: 0.6254153003970168                                                       
SCORE: 0.6208923611975157                                                       
SCORE: 0.6304480758845175                                                       
SCORE: 0.6356901609487834                                                       
SCORE: 0.6205587362509906       

SCORE: 0.6191646894889964                                                        
SCORE: 0.62311959083039                                                          
SCORE: 0.6286717272185294                                                        
SCORE: 0.6287833124859088                                                        
SCORE: 0.6202186236572979                                                        
SCORE: 0.6365206206177951                                                        
SCORE: 0.6190851486437958                                                        
SCORE: 0.6204242319152187                                                        
SCORE: 0.6261195221622442                                                        
100%|██████████| 50/50 [00:04<00:00, 12.24trial/s, best loss: 0.6190851486437958]
The best hyperparameters are :  

{'C': 0.27536592546443595}
% different predictions w\o Demo: 0.1875
Mean absolute % difference w\o Demo: 0.007912209735263208
RMSE Difference w\o 

In [28]:
np.mean(mean_abs_differences).round(2),np.std(mean_abs_differences).round(2)

(0.11, 0.05)

In [29]:
f1(y_test,y_pred,average="macro"),f1(y_test,y_pred_notdemo,average="macro")

(0.7918797342579428, 0.7528299262967599)